<a href="https://colab.research.google.com/github/RenaAbbasova/Projects/blob/master/Tesla_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Problem Statement and Agenda:

One of the applications of Time Series Forecasting is to predict opening stock prices, closing stock prices, and the volume of stocks to be traded, among others. In this dataset, we aim to forecast the future behavior of the stock market, focusing on the average stock price data of Tesla spanning from 2010 to 2024.

For this case study, we will divide our data into training and test sets, build our models on the training data, forecast for the test data timestamps, and then evaluate using the Root Mean Squared Error (RMSE) model evaluation metric.

The following topics will be covered in this case study:

Exploratory Data Analysis
ARIMA/SARIMA models (with and without exogenous variables)
Facebook Prophet Model
LSTM Model (Deep Learning Model)


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
sns.set()
%matplotlib inline
from pandas import Series
from numpy import log
import plotly.express as px # high level interface
import plotly.graph_objects as go # lower level interface

In [ ]:
# read the data
data=pd.read_excel('/content/drive/MyDrive/data/TESLA.xlsx')

In [ ]:
data

In [ ]:
data1=data.copy()

In [ ]:
data1.set_index('Date',inplace=True)

In [ ]:
data1.head()

In [ ]:
df = data.copy()

#EDA

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
# there is not duplicated and  null values
# we going to keep only 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP' variables

df.drop(['Adj Close'], axis=1, inplace=True)

In [ ]:
df.head()

# DURBIN - WATSON TEST

In [ ]:
 #Durbin-Watson test going to be applied to residuals from time series models to diagnose autocorrelation.

In [ ]:
import statsmodels.api as sm
print(sm.stats.durbin_watson(df['VWAP']))
print(sm.stats.durbin_watson(df['Open']))
print(sm.stats.durbin_watson(df['High']))
print(sm.stats.durbin_watson(df['Low']))
print(sm.stats.durbin_watson(df['Close']))
print(sm.stats.durbin_watson(df['Volume']))

In [ ]:
# There is strong evidence of positive autocorrelation in the residuals of the 'VWAP', 'Open', 'High',
# 'Low', and 'Close' variables.

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
start_date = df['Date'].min()
end_date = df['Date'].max()
print(start_date)
print(end_date)

In [ ]:
df.shape

In [ ]:
# check frequency of VWAP variable
df.VWAP.plot(figsize=(20,5),title='VWAP')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x=data1.index.year, y =df['VWAP'],palette='pastel')
plt.grid()

In [ ]:
# We can observe from the graph how the volume-weighted average price has been increasing since 2010,
# with the best year for selling being 2021.

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x=data1.index.month_name(), y =df['VWAP'],palette='pastel')
plt.grid()

In [ ]:
# Selling in November was good, as the prices were high.
# For buying in all other months, the prices remained similar.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose


In [ ]:
# Decompose the time series into trend, seasonal, cyclical, and irregular components.
decompose = seasonal_decompose(df['VWAP'], model='additive', period=365)

# Extract the components
trend = decompose.trend
seasonal = decompose.seasonal
cyclical = df['VWAP']  - trend - seasonal
irregular = decompose.resid

In [ ]:
# Plot the components
plt.subplot(411)
plt.plot(df['VWAP'], label='Original')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal, label='Seasonality')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(irregular, label='Residuals')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

# Q-Q plot

In [ ]:
import scipy.stats
import pylab

In [ ]:
scipy.stats.probplot(df.VWAP, plot=pylab)
plt.title("QQ plot for Volume Waighted Price Average")
pylab.show()

In [ ]:
import scipy.stats as stats

# Replace 'desired_distribution' with the name of the distribution you want to test against (e.g., 'expon', 'gamma', 'uniform', etc.)
stats.probplot(df['VWAP'], dist='expon', plot=plt)
plt.title("QQ plot for Volume Weighted Price Average")
plt.show()


In [ ]:
trend=decompose.trend
seasonality=decompose.seasonal
residual=decompose.resid

print('Trend', '\n', trend.head(12), '\n')

print('Seasonality', '\n', seasonality.head(12), '\n')

print('Residual', '\n', residual.head(12), '\n')

In [ ]:
# Creating new column
df['Time_Stamp']=pd.DataFrame(df, columns=['Date'])
df['Time_Stamp'] = pd.to_datetime(df['Time_Stamp'])
# set index column 'Time_Stamp'
df_final = df.set_index('Time_Stamp')


In [ ]:
df_final.head()

# Spit the data into training and test before building Time Series Forecasting

In [ ]:
df_final['Month'] = df_final.index.month
df_final['Year'] = df_final.index.year

In [ ]:
df_final.head()

In [ ]:
df_final.index.max()

In [ ]:
df_final.index.min()

In [ ]:
# split into training and tesing data
train_df = df_final[pd.to_datetime(df_final['Date']) < pd.to_datetime('2019-10-04')]

test_df = df_final[pd.to_datetime(df_final['Date']) >= pd.to_datetime('2019-10-04')]

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_final = train_df[['VWAP']]
test_final = test_df[['VWAP']]

In [ ]:
train_final.head()

In [ ]:
test_final.head()

# Augmented Dickey Fuller Test

In [ ]:
# To check time series stationary or not

from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(train_final['VWAP'])

In [ ]:
def check_adftest(timeseries):
  result=adfuller(timeseries)
  print('Augmented Dickey Fuller Test')
  labels = ['ADF test', 'P-value','#Lags', 'No of observation']

  for i, j in zip(result,labels):
    print(j + '------->'+str(i))

  if result[1] <= 0.05:
    print('Strong evidence against Null Hypothesis and my time series is Stationary')

  else:
    print('Weak Evidence against Null Hypothesis and my time series is Non-Stationary')

In [ ]:
check_adftest(train_final['VWAP'])

In [ ]:
# as time series is non - stationary
# we aply Differencing technique

In [ ]:
# Difference the time series and plot
diff_data = train_final['VWAP'].diff().dropna()
plt.plot(diff_data)
plt.title('Differenced Time Series')
plt.show()




In [ ]:
check_adftest(diff_data)

In [ ]:
# This code generates combinations of parameters for a SARIMA (Seasonal Autoregressive Integrated Moving Average) model.
# SARIMA models are extensions of the ARIMA model that include seasonal components.

In [ ]:
import itertools
p = q = range(0,3)

d = range(0,1)

pdq = list(itertools.product(p, d, q)) # Trend

model_pdq = [(x[0], x[1], x[2], 5) for x in list(itertools.product(p,d,q))] # Seasonality

print('Example of combination for model...........')
print('Model : {}{}'.format(pdq[1],model_pdq[1]))
print('Model : {}{}'.format(pdq[0],model_pdq[0]))
print('Model : {}{}'.format(pdq[2],model_pdq[2]))
print('Model : {}{}'.format(pdq[1],model_pdq[2]))

# Build SARIMAX model - Seasonality Autoregressive integrated moving average with external factor


SARIMAX is a statistical model used for time series forecasting. It's an extension of the ARIMA model that incorporates additional features to handle seasonality and exogenous variables (variables external to the model that can influence the time series being analyzed).

In [ ]:
ex_train =  train_df[['Open', 'High', 'Low', 'Close', 'Volume']]
ex_test = test_df[['Open', 'High', 'Low', 'Close', 'Volume']]

In [ ]:
ex_train.head()

In [ ]:
df_obj=pd.DataFrame(columns=['param','seasonal','AIC'])
df_obj

In [ ]:
# Lets build the model
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA


# Initialize an empty list to store results
results = []


for param in pdq:
  for param_seasonal in model_pdq:
    model = sm.tsa.statespace.SARIMAX(train_final['VWAP'], exog=ex_train, order=param, seasonal_order=param_seasonal,
                                      enforce_stationarity=False, enforce_invertibility=False)
    result_SARIMAX = model.fit()
    print('SARIMAX{}{} - AIC:{}'.format(param,param_seasonal,result_SARIMAX.aic))


# Append the results to the list
    results.append({'param': param, 'seasonal': param_seasonal, 'AIC': result_SARIMAX.aic})

# Convert the list of dictionaries to a DataFrame
df_obj = pd.DataFrame(results)
    #df_obj=df_obj.append({'param': param, 'seasonal': param_seasonal,
                            #  'AIC': result_SARIMAX.aic}, ignore_index=True)

In [ ]:
df_obj.sort_values(by=['AIC'])

In [ ]:
#(0, 0, 0)	(0, 0, 0, 5)	-47978.760521
model = sm.tsa.statespace.SARIMAX(train_final['VWAP'], exog=ex_train, order=(0, 0, 0),
                                   seasonal_order=(0, 0, 0, 5),
                                   enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()
print(result.summary())



##### In the SARIMAX model summary output , the 'Open' and 'Volume' variables have coefficients close to zero and p-values greater than 0.05. This indicates that these variables are not statistically significant predictors for the dependent variable ('VWAP') in the model.

In [ ]:
result.plot_diagnostics(figsize=(16,8))
plt.show()

In [ ]:
predict_SARIMAX = result.get_forecast(steps=len(test_df), exog=ex_test)
predict_SARIMAX.predicted_mean

In [ ]:
from sklearn.metrics import *
import math
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
rmse = sqrt(mean_squared_error(test_final.VWAP, predict_SARIMAX.predicted_mean, squared=False))
print(rmse)

##### An RMSE (Root Mean Square Error) value of 0.00014617368289626304 indicates that the SARIMAX model's predictions are very close to the actual values on average.

In [ ]:
plt.plot(train_final, label='Training Data')
plt.plot(test_final, label='Test Data')
plt.plot(test_final.index, predict_SARIMAX.predicted_mean, label='predicted Model -SARIMAX')
plt.legend(loc='best')
plt.grid();

# Sarima Model

In [ ]:
model = sm.tsa.statespace.SARIMAX(train_final['VWAP'],order=(1,1,2),
                                   seasonal_order=(1,0,2,5),
                                   enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()
print(result.summary())

In [ ]:
predict_SARIMA = result.get_forecast(steps=len(test_df), exog=ex_test)
predict_SARIMA.predicted_mean

In [ ]:
rmse = sqrt(mean_squared_error(test_final.VWAP, predict_SARIMA.predicted_mean, squared=False))
print(rmse)

In [ ]:
plt.plot(train_final, label='Training Data')
plt.plot(test_final, label='Test Data')
plt.plot(test_final.index, predict_SARIMAX.predicted_mean, label='predicted Model -SARIMAX')
plt.plot(test_final.index, predict_SARIMA.predicted_mean, label='predicted Model -SARIMA')
plt.legend(loc='best')
plt.grid();

##### The SARIMAX model performs close to the actual values and provides satisfactory forecast accuracy, it suggests that incorporating exogenous variables has helped capture important information in our time series data.

# LSTM Model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense




## **Conclusion**

In this project, we focused on predicting the VWAP using 14 year's worth of time series data. Throughout the project, we followed a systematic approach to build and evaluate time series models.

We began by reading and preprocessing the data, ensuring that it was in a suitable format for analysis. We then conducted exploratory data analysis (EDA) to gain insights into the patterns, trends, and seasonality present in the data.

To proceed with modeling, we first checked the stationarity of the time series. Since the data was non - stationary, we performed differencing technique.

We applied Durbin-Watson test to residuals from time series models to diagnose autocorrelation.

We built  models like Sarima and Sarimax. The performance of these models was evaluated based on both the p-value and root mean squared error (RMSE).

After evaluating the models, we found that Sarimax model outperformed the Sarima model in terms of predictive accuracy.


